### Retinal Disease Classification using different Deep Learning Models

Import required modules

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import cv2

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator

Loading Data

In [2]:
# Datasets
train_labels_path = 'Training_Set/RFMiD_Training_Labels.csv'
test_labels_path = 'Test_Set/RFMiD_Testing_Labels.csv'
validation_labels_path = 'Evaluation_Set/RFMiD_Validation_Labels.csv'

train_path = 'Training_Set/Training'
test_path = 'Test_Set/Test'
validation_path = 'Evaluation_Set/Validation'

In [3]:
# Loading labels
train_labels = pd.read_csv(train_labels_path)
test_labels = pd.read_csv(test_labels_path)
val_labels = pd.read_csv(validation_labels_path)

In [4]:
print("Train shape:", train_labels.shape)
print("Test shape:", test_labels.shape)
print("Validation shape:", val_labels.shape)

Train shape: (1920, 47)
Test shape: (640, 47)
Validation shape: (640, 47)


In [5]:
# to dislpay all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [6]:
print(train_labels.head())

   ID  Disease_Risk  DR  ARMD  MH  DN  MYA  BRVO  TSLN  ERM  LS  MS  CSR  ODC  \
0   1             1   1     0   0   0    0     0     0    0   0   0    0    0   
1   2             1   1     0   0   0    0     0     0    0   0   0    0    0   
2   3             1   1     0   0   0    0     0     0    0   0   0    0    0   
3   4             1   0     0   1   0    0     0     0    0   0   0    0    1   
4   5             1   1     0   0   0    0     0     0    0   1   0    0    0   

   CRVO  TV  AH  ODP  ODE  ST  AION  PT  RT  RS  CRS  EDN  RPEC  MHL  RP  CWS  \
0     0   0   0    0    0   0     0   0   0   0    0    0     0    0   0    0   
1     0   0   0    0    0   0     0   0   0   0    0    0     0    0   0    0   
2     0   0   0    0    0   0     0   0   0   0    0    0     0    0   0    0   
3     0   0   0    0    0   0     0   0   0   0    0    0     0    0   0    0   
4     0   0   0    0    0   0     0   0   0   0    0    0     0    0   0    0   

   CB  ODPM  PRH  MNF  HR 

In [7]:
# Exploratory Data Analysis
disease_counts = train_labels.iloc[:, 2:].sum().sort_values(ascending=False)
#using plotly
fig = px.bar(disease_counts, title="Disease Distribution in Training Set") 
fig.show()

In [8]:
# Preprocessing the data
# function to load the images and labels
def load_images_and_labels(image_dir, labels_df):
    images = []
    labels = []
    for index, row in labels_df.iterrows():
        img_path = os.path.join(image_dir, f"{row['ID']}.png")
        if os.path.exists(img_path):
            img = cv2.imread(img_path)
            # Resizing images to 224x224
            img = cv2.resize(img, (224, 224))
            images.append(img)
            labels.append(row['Disease_Risk'])
    return np.array(images), np.array(labels)

# Load images and labels
x_train, y_train = load_images_and_labels(train_path, train_labels)
x_test, y_test = load_images_and_labels(test_path, test_labels)
x_val, y_val = load_images_and_labels(validation_path, val_labels)

In [9]:
# Normalizing the images
x_train = x_train / 255.0
x_val = x_val / 255.0
x_test = x_test / 255.0

In [10]:
# Image Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

Defining the model

In [11]:
warnings.filterwarnings('ignore')
# Using Convolutional Neural Network

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Train the model

In [12]:
warnings.filterwarnings('ignore')

history = model.fit(
    datagen.flow(x_train, y_train, batch_size=32),
    validation_data=(x_val, y_val),
    epochs=20
)

Epoch 1/20


60/60 [==============================] - 240s 4s/step - loss: 0.5674 - accuracy: 0.7755 - val_loss: 0.4857 - val_accuracy: 0.7906
Epoch 2/20
60/60 [==============================] - 186s 3s/step - loss: 0.4651 - accuracy: 0.7911 - val_loss: 0.4788 - val_accuracy: 0.7906
Epoch 3/20
60/60 [==============================] - 175s 3s/step - loss: 0.4744 - accuracy: 0.7927 - val_loss: 0.4703 - val_accuracy: 0.7906
Epoch 4/20
60/60 [==============================] - 179s 3s/step - loss: 0.4638 - accuracy: 0.7911 - val_loss: 0.4463 - val_accuracy: 0.7906
Epoch 5/20
60/60 [==============================] - 193s 3s/step - loss: 0.4542 - accuracy: 0.7911 - val_loss: 0.4398 - val_accuracy: 0.7906
Epoch 6/20
60/60 [==============================] - 161s 3s/step - loss: 0.4504 - accuracy: 0.7911 - val_loss: 0.4484 - val_accuracy: 0.7906
Epoch 7/20
60/60 [==============================] - 181s 3s/step - loss: 0.4501 - accuracy: 0.7917 - val_loss: 0.4482 - val_accuracy: 0.7906
Epoch 8/20


Evaluate the model

In [13]:
y_pred = (model.predict(x_test) > 0.5).astype("int32")

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

20/20 [==============================] - 10s 357ms/step
Accuracy: 0.871875
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.60      0.66       134
           1       0.90      0.94      0.92       506

    accuracy                           0.87       640
   macro avg       0.82      0.77      0.79       640
weighted avg       0.87      0.87      0.87       640



In [14]:
cm = confusion_matrix(y_test, y_pred)
fig = px.imshow(cm, text_auto=True, title="Confusion Matrix")
fig.show()

Save the model

In [15]:
model.save('cnn_model.h5')